## Define a Class Object

In [5]:
class MyFirstClass:
    favorite_number = 41
    def print_number(number):
        value= 63
        print(number)

***Attribute Resolution on Class Objects***
---
Our `class` object also has `attributes` (the ones we defined in its body!) that we can access with a new-ish syntax - asking Python for `some_object.some_attribute` instructs the name `some_attribute` to be resolved on the `some_object` object. For us, this looks like

In [6]:
print(type(MyFirstClass.__dict__))  # something that acts a lot like a normal dictionary
print('favorite_number' in MyFirstClass.__dict__)  # True
print('greet' in MyFirstClass.__dict__)  # True

<class 'mappingproxy'>
True
False


In [7]:
MyFirstClass.__dict__

mappingproxy({'__module__': '__main__',
              'favorite_number': 41,
              'print_number': <function __main__.MyFirstClass.print_number(number)>,
              '__dict__': <attribute '__dict__' of 'MyFirstClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyFirstClass' objects>,
              '__doc__': None})

In [9]:
getattr(MyFirstClass, 'favorite_number')

41

In [18]:
class Dog:
    def __init__(self, name, tricks=None ):
        self.name = name
        if tricks : 
            self.tricks = tricks 
        else: 
            self.tricks = set() 
    
    def teach(self, trick):
        self.tricks.add(trick)
        
# Change the broken code above so that the following lines work:

buddy = Dog('Buddy')
pascal = Dog('Pascal')
kimber = Dog('Kimber', tricks={'lie down', 'shake'})
buddy.teach('sit')
pascal.teach('fetch')
buddy.teach('roll over')
kimber.teach('fetch')
print(buddy.tricks)  # {'roll over', 'sit'}
print(pascal.tricks)  # {'fetch'}
print(kimber.tricks)  # {'fetch', 'shake', 'lie down'}

{'sit', 'roll over'}
{'fetch'}
{'fetch', 'shake', 'lie down'}


In [24]:
# This one's a bit different, representing an unusual (and honestly,
# not recommended) strategy for tracking users that sign up for a service.

class User:
    # An (intentionally shared) collection storing users who sign up for some hypothetical service.
    # There's only one set of members, so it lives at the class level!
    members = set()
    def __init__(self, name , members = set()):
        self.name = name
        self.members = members  # Not signed up to begin with.
    def sign_up(self):
        User.members.add(self.name)

# Change the code above so that the following lines work:
# 
sarah = User('sarah')
heather = User('heather')
cristina = User('cristina')
print(User.members)  # set()
heather.sign_up()
cristina.sign_up()
print(User.members)  # {'heather', 'cristina'}

set()
{'cristina', 'heather'}


In [15]:
id(pascal.tricks)

2403939576736